In [65]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
import dmc
from process import *

In [67]:
evaluation_sets = ['rawMirrored', 'rawLinearSample']
data = processed_data()

In [68]:
train, test = split_data_by_id(data, 'rawMirrored')

/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [70]:
len(train), len(test)

(947782, 236498)

In [71]:
ensemble = dmc.ensemble.Ensemble(train, test)

In [72]:
split = ensemble.splits['unknown_articleID-known_customerID-unknown_voucherID-known_productGroup']

In [73]:
import csv
file_name = 'ArticleProductgroupCustomer.csv'
with open('/Users/Fabian/Downloads/documents-export-2016-05-11/' + 
          file_name) as csv_file:
    for index, row in csv.reader(csv_file, delimiter=','):
        print(index, row)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Fabian/Downloads/documents-export-2016-05-11/ArticleProductgroupCustomer.csv'

In [15]:
ensemble.transform??

In [ ]:
ensemble.classify??

In [63]:
ensemble.transform(binary_target=True, scalers=[dmc.transformation.normalize_raw_features]*10, 
                   ignore_features=[['orderID']] * 10)

KeyError: 0